In [ ]:
from pathlib import Path
import re

base_path = Path("/content/drive/MyDrive/NLP PROJECT")
train_src = base_path / "train_en.txt"
train_tgt = base_path / "train_ar.txt"
val_src = base_path / "dev.en"
val_tgt = base_path / "dev.ar"


def load_parallel_data(src_path, tgt_path):
    with open(src_path, encoding="utf-8") as f_src, open(tgt_path, encoding="utf-8") as f_tgt:
        src_lines = f_src.readlines()
        tgt_lines = f_tgt.readlines()
    return [src.strip() for src in src_lines], [tgt.strip() for tgt in tgt_lines]

train_en, train_ar = load_parallel_data(train_src, train_tgt)
val_en, val_ar = load_parallel_data(val_src, val_tgt)

def preprocess_en(texts):
    return [re.sub(r'\s+', ' ', re.sub(r'[^\w\s]', '', t.lower().strip())) for t in texts]

def preprocess_ar(texts):
    def normalize(text):
        text = re.sub(r"[إأآا]", "ا", text)
        text = re.sub(r"ى", "ي", text)
        text = re.sub(r"ؤ", "ء", text)
        text = re.sub(r"ئ", "ء", text)
        text = re.sub(r"ة", "ه", text)
        text = re.sub(r"[^\w\s]", '', text)
        text = re.sub(r"\s+", " ", text)
        text = re.sub(r"ـ", "", text)
        return text.strip()
    return [normalize(t) for t in texts]

train_en, train_ar = load_parallel_data(train_src, train_tgt)
val_en, val_ar = load_parallel_data(val_src, val_tgt)

train_en = preprocess_en(train_en)
train_ar = preprocess_ar(train_ar)
val_en = preprocess_en(val_en)
val_ar = preprocess_ar(val_ar)

In [ ]:
from transformers import MarianTokenizer, MarianMTModel, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset
import torch

# Choose pre-trained model (you can also reverse for ar→en)
model_name = "Helsinki-NLP/opus-mt-en-ar"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Prepare HuggingFace Dataset
def encode_data(src_texts, tgt_texts, tokenizer, max_length=128):
    model_inputs = tokenizer(src_texts, max_length=max_length, truncation=True, padding="max_length", return_tensors="pt")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(tgt_texts, max_length=max_length, truncation=True, padding="max_length", return_tensors="pt")["input_ids"]
    model_inputs["labels"] = labels
    return model_inputs



/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
train_dataset = Dataset.from_dict(encode_data(train_en, train_ar, tokenizer))
val_dataset = Dataset.from_dict(encode_data(val_en, val_ar, tokenizer))

# Training args
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/NLP PROJECT/results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    fp16=torch.cuda.is_available()
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
<ipython-input-10-e8f9e3f2f7b8>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,0.468000,0.517828
2,0.440600,0.512432


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62801]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=53022, training_loss=0.46525635429178863, metrics={'train_runtime': 6021.0689, 'train_samples_per_second': 70.448, 'train_steps_per_second': 8.806, 'total_flos': 1.437866945150976e+16, 'train_loss': 0.46525635429178863, 'epoch': 2.0})

In [ ]:
from evaluate import load
import numpy as np

bleu = load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]  # nested list for sacrebleu

    return {"bleu": bleu.compute(predictions=decoded_preds, references=decoded_labels)["score"]}


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


<ipython-input-20-3482aa9d5b4d>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
eval_results = trainer.evaluate()
print("Evaluation BLEU score:", eval_results["eval_bleu"])


Evaluation BLEU score: 18.30567141538107


In [ ]:
def translate_sentences(sentences, tokenizer, model, max_length=128):
    model.eval()
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=max_length, num_beams=4)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)


In [ ]:
custom_en_sentences = [
    "This is our NLP project",
    "We're learning how to build a machine translation model.",
    "Welcome to our Simple model, hope you like it!.",
    "Faculty of computers and artificial intelligence Helwan university graduates 2025!."
]

# Translate from English to Arabic
translated_ar = translate_sentences(custom_en_sentences, tokenizer, model)

# Print results
for src, pred in zip(custom_en_sentences, translated_ar):
    print(f"EN: {src}\nAR: {pred}\n")


EN: This is our NLP project
AR: هذا هو مشروعنا NLP.

EN: We're learning how to build a machine translation model.
AR: نحن نتعلم كيفية بناء نموذج الترجمة الآلية.

EN: Welcome to our Simple model, hope you like it!.
AR: مرحبا بكم في نموذجنا البسيط, أتمنى أن تعجبكم!

EN: Faculty of computers and artificial intelligence Helwan university graduates 2025!.
AR: كلية الكمبيوترات والذكاء الاصطناعي خريجي جامعة هلوان 2025!



In [ ]:
# Define a directory to save the model
save_directory = "/content/drive/MyDrive/NLP PROJECT/fine-tuned-en-ar"

# Save model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)


('/content/drive/MyDrive/NLP PROJECT/fine-tuned-en-ar/tokenizer_config.json',
 '/content/drive/MyDrive/NLP PROJECT/fine-tuned-en-ar/special_tokens_map.json',
 '/content/drive/MyDrive/NLP PROJECT/fine-tuned-en-ar/vocab.json',
 '/content/drive/MyDrive/NLP PROJECT/fine-tuned-en-ar/source.spm',
 '/content/drive/MyDrive/NLP PROJECT/fine-tuned-en-ar/target.spm',
 '/content/drive/MyDrive/NLP PROJECT/fine-tuned-en-ar/added_tokens.json')